# Exercise 4: Agentic Systems with LangGraph

Build intelligent agent systems that can reason, use tools, and execute multi-step workflows using LangGraph.

**Learning Objectives:**
- Understand agentic AI architectures
- Build graph-based agent workflows with LangGraph
- Implement tool use and function calling
- Create reasoning and planning loops
- Manage state across agent execution
- Build practical multi-step agents

## Part 1: Setup and Introduction

In [ ]:
!pip install langgraph langchain langchain-community transformers torch python-dotenv -q

In [ ]:
import torch
from typing import TypedDict, Annotated, Sequence
from langgraph.graph import StateGraph, END
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
import operator
import json

print("Setup complete!")

## Part 2: Understanding Agentic Systems

Agentic AI systems can:
- **Reason**: Think through problems step-by-step
- **Act**: Use tools to interact with the environment
- **Observe**: Process results and adapt
- **Plan**: Break down complex tasks into subtasks

Key components:
1. **LLM Brain**: Makes decisions
2. **Tools**: Actions the agent can take
3. **Memory/State**: Tracks conversation and context
4. **Control Flow**: Determines next actions

In [ ]:
# Simple example: Agent that decides whether to use a calculator

def simple_agent_example():
    """
    A basic agent that:
    1. Receives a query
    2. Decides if it needs a tool
    3. Uses tool if needed
    4. Returns final answer
    """
    
    query = "What is 15 * 27?"
    
    # Step 1: Reasoning
    needs_calculator = True  # Agent decides this requires calculation
    
    if needs_calculator:
        # Step 2: Tool use
        result = 15 * 27
        
        # Step 3: Formulate response
        answer = f"Using calculator: 15 * 27 = {result}"
    else:
        answer = "I can answer that directly without tools."
    
    return answer

print("Simple Agent Example:")
print(simple_agent_example())
print("\nNow let's build this properly with LangGraph!")

## Part 3: LangGraph Fundamentals

LangGraph represents agent workflows as graphs:
- **Nodes**: Processing steps (LLM calls, tool use, logic)
- **Edges**: Transitions between nodes
- **State**: Shared data structure passed between nodes
- **Conditional Edges**: Dynamic routing based on state

In [ ]:
# Define State: What information flows through the graph
class AgentState(TypedDict):
    """State that gets passed between nodes."""
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next_action: str

# TODO: Create a simple 3-node graph

def input_node(state: AgentState) -> AgentState:
    """Process initial input."""
    print("📥 Input Node: Processing user query")
    return {**state, "next_action": "think"}

def thinking_node(state: AgentState) -> AgentState:
    """Agent reasoning step."""
    print("🤔 Thinking Node: Analyzing the problem")
    messages = state["messages"]
    last_message = messages[-1].content if messages else ""
    
    # Simple logic: check if query contains math
    if any(op in last_message for op in ["+", "-", "*", "/"]):
        return {**state, "next_action": "calculate"}
    else:
        return {**state, "next_action": "respond"}

def calculate_node(state: AgentState) -> AgentState:
    """Perform calculation."""
    print("🔢 Calculate Node: Using calculator tool")
    # In real agent, this would call a tool
    return {**state, "next_action": "respond"}

def respond_node(state: AgentState) -> AgentState:
    """Generate final response."""
    print("💬 Respond Node: Generating final answer")
    return {**state, "next_action": "end"}

# Build the graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("input", input_node)
workflow.add_node("think", thinking_node)
workflow.add_node("calculate", calculate_node)
workflow.add_node("respond", respond_node)

# Add edges
workflow.set_entry_point("input")
workflow.add_edge("input", "think")

# Conditional routing from thinking node
workflow.add_conditional_edges(
    "think",
    lambda state: state["next_action"],
    {
        "calculate": "calculate",
        "respond": "respond"
    }
)

workflow.add_edge("calculate", "respond")
workflow.add_edge("respond", END)

# Compile the graph
app = workflow.compile()

print("✅ Graph built successfully!")
print("\nGraph structure:")
print("input → think → [calculate OR respond] → respond → END")

In [ ]:
# Test the graph
print("\nTest 1: Math query")
print("=" * 50)
result = app.invoke({
    "messages": [HumanMessage(content="What is 15 + 27?")],
    "next_action": "start"
})

print("\n\nTest 2: Non-math query")
print("=" * 50)
result = app.invoke({
    "messages": [HumanMessage(content="What is the capital of France?")],
    "next_action": "start"
})

## Part 4: Implementing Tools

Tools give agents the ability to interact with the world.

In [ ]:
# TODO: Define custom tools using @tool decorator

@tool
def calculator(expression: str) -> str:
    """Evaluates a mathematical expression safely.
    
    Args:
        expression: A mathematical expression like '2 + 2' or '15 * 27'
    
    Returns:
        The result of the calculation as a string
    """
    try:
        # Safe evaluation
        result = eval(expression, {"__builtins__": {}}, {})
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def search_knowledge_base(query: str) -> str:
    """Searches a knowledge base for information.
    
    Args:
        query: The search query
    
    Returns:
        Relevant information from the knowledge base
    """
    # Simulated knowledge base
    knowledge = {
        "python": "Python is a high-level programming language.",
        "transformers": "Transformers are neural network architectures based on self-attention.",
        "langgraph": "LangGraph is a framework for building agent workflows."
    }
    
    query_lower = query.lower()
    for key, value in knowledge.items():
        if key in query_lower:
            return value
    
    return "No information found in knowledge base."

@tool
def get_current_time() -> str:
    """Gets the current time.
    
    Returns:
        Current time as a string
    """
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# List of available tools
tools = [calculator, search_knowledge_base, get_current_time]

print("Available tools:")
for tool_obj in tools:
    print(f"  - {tool_obj.name}: {tool_obj.description}")

In [ ]:
# Test the tools
print("Testing calculator:")
print(calculator.invoke({"expression": "15 * 27"}))

print("\nTesting knowledge base:")
print(search_knowledge_base.invoke({"query": "What are transformers?"}))

print("\nTesting time:")
print(get_current_time.invoke({}))

## Part 5: Building a Tool-Using Agent

Create an agent that can decide which tool to use based on the query.

In [ ]:
# Enhanced state with tool calls
class ToolAgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    tool_to_use: str
    tool_input: str
    tool_output: str

def analyze_query_node(state: ToolAgentState) -> ToolAgentState:
    """Determine what tool (if any) is needed."""
    messages = state["messages"]
    last_message = messages[-1].content if messages else ""
    
    print(f"🔍 Analyzing query: {last_message}")
    
    # Simple rule-based tool selection
    tool_to_use = "none"
    tool_input = ""
    
    if any(op in last_message for op in ["+", "-", "*", "/", "calculate"]):
        tool_to_use = "calculator"
        # Extract expression (simple approach)
        import re
        match = re.search(r'[\d\s+\-*/().]+', last_message)
        tool_input = match.group(0).strip() if match else last_message
    
    elif "time" in last_message.lower():
        tool_to_use = "get_current_time"
        tool_input = ""
    
    elif any(word in last_message.lower() for word in ["what is", "what are", "tell me about"]):
        tool_to_use = "search_knowledge_base"
        tool_input = last_message
    
    print(f"  → Selected tool: {tool_to_use}")
    
    return {
        **state,
        "tool_to_use": tool_to_use,
        "tool_input": tool_input,
        "tool_output": ""
    }

def execute_tool_node(state: ToolAgentState) -> ToolAgentState:
    """Execute the selected tool."""
    tool_name = state["tool_to_use"]
    tool_input = state["tool_input"]
    
    print(f"🔧 Executing tool: {tool_name}")
    
    # Find and execute the tool
    tool_map = {t.name: t for t in tools}
    
    if tool_name in tool_map:
        if tool_name == "calculator":
            output = tool_map[tool_name].invoke({"expression": tool_input})
        elif tool_name == "search_knowledge_base":
            output = tool_map[tool_name].invoke({"query": tool_input})
        else:
            output = tool_map[tool_name].invoke({})
    else:
        output = "No tool needed."
    
    print(f"  → Tool output: {output}")
    
    return {**state, "tool_output": output}

def generate_response_node(state: ToolAgentState) -> ToolAgentState:
    """Generate final response using tool output."""
    tool_output = state.get("tool_output", "")
    original_query = state["messages"][-1].content
    
    # Create response
    if tool_output:
        response = f"Based on the query '{original_query}', {tool_output}"
    else:
        response = f"I don't have specific information about: {original_query}"
    
    print(f"💬 Response: {response}")
    
    return {
        **state,
        "messages": state["messages"] + [AIMessage(content=response)]
    }

# Build tool-using agent graph
tool_workflow = StateGraph(ToolAgentState)

tool_workflow.add_node("analyze", analyze_query_node)
tool_workflow.add_node("execute_tool", execute_tool_node)
tool_workflow.add_node("respond", generate_response_node)

tool_workflow.set_entry_point("analyze")

# Conditional: use tool or skip to response
tool_workflow.add_conditional_edges(
    "analyze",
    lambda state: "use_tool" if state["tool_to_use"] != "none" else "skip_tool",
    {
        "use_tool": "execute_tool",
        "skip_tool": "respond"
    }
)

tool_workflow.add_edge("execute_tool", "respond")
tool_workflow.add_edge("respond", END)

tool_agent = tool_workflow.compile()

print("\n✅ Tool-using agent built!")

In [ ]:
# Test the tool-using agent
test_queries = [
    "What is 123 * 456?",
    "What time is it?",
    "Tell me about transformers",
    "Hello, how are you?"
]

for query in test_queries:
    print("\n" + "=" * 70)
    print(f"Query: {query}")
    print("=" * 70)
    
    result = tool_agent.invoke({
        "messages": [HumanMessage(content=query)],
        "tool_to_use": "",
        "tool_input": "",
        "tool_output": ""
    })
    
    print()

## Part 6: Multi-Step Reasoning Agent

Build an agent that can break down complex tasks into multiple steps.

In [ ]:
# TODO: Create a multi-step planning agent

class PlanningAgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    plan: list[str]
    current_step: int
    step_results: list[str]
    final_answer: str

def create_plan_node(state: PlanningAgentState) -> PlanningAgentState:
    """Break down the query into steps."""
    query = state["messages"][-1].content
    
    print(f"📋 Creating plan for: {query}")
    
    # Simple planning logic
    plan = []
    
    if "calculate" in query.lower() and "then" in query.lower():
        # Multi-step calculation
        parts = query.lower().split("then")
        for i, part in enumerate(parts, 1):
            plan.append(f"Step {i}: {part.strip()}")
    else:
        # Single step
        plan = [f"Step 1: {query}"]
    
    print(f"  Plan:")
    for step in plan:
        print(f"    - {step}")
    
    return {
        **state,
        "plan": plan,
        "current_step": 0,
        "step_results": [],
        "final_answer": ""
    }

def execute_step_node(state: PlanningAgentState) -> PlanningAgentState:
    """Execute current step of the plan."""
    current_step = state["current_step"]
    plan = state["plan"]
    
    if current_step >= len(plan):
        return state
    
    step = plan[current_step]
    print(f"\n⚙️  Executing: {step}")
    
    # Execute the step (simplified)
    result = f"Completed: {step}"
    
    step_results = state["step_results"] + [result]
    
    return {
        **state,
        "current_step": current_step + 1,
        "step_results": step_results
    }

def check_completion_node(state: PlanningAgentState) -> str:
    """Check if all steps are complete."""
    if state["current_step"] >= len(state["plan"]):
        return "finalize"
    return "continue"

def finalize_node(state: PlanningAgentState) -> PlanningAgentState:
    """Combine results into final answer."""
    print("\n✨ Finalizing answer...")
    
    final_answer = "Completed all steps:\n" + "\n".join(state["step_results"])
    
    return {
        **state,
        "final_answer": final_answer,
        "messages": state["messages"] + [AIMessage(content=final_answer)]
    }

# Build planning agent
planning_workflow = StateGraph(PlanningAgentState)

planning_workflow.add_node("plan", create_plan_node)
planning_workflow.add_node("execute", execute_step_node)
planning_workflow.add_node("finalize", finalize_node)

planning_workflow.set_entry_point("plan")
planning_workflow.add_edge("plan", "execute")

planning_workflow.add_conditional_edges(
    "execute",
    check_completion_node,
    {
        "continue": "execute",
        "finalize": "finalize"
    }
)

planning_workflow.add_edge("finalize", END)

planning_agent = planning_workflow.compile()

print("✅ Planning agent built!")

In [ ]:
# Test planning agent
query = "Calculate 10 + 5 then multiply the result by 3"

print(f"Complex Query: {query}")
print("=" * 70)

result = planning_agent.invoke({
    "messages": [HumanMessage(content=query)],
    "plan": [],
    "current_step": 0,
    "step_results": [],
    "final_answer": ""
})

print("\n" + "=" * 70)
print("Final Answer:")
print(result["final_answer"])

## Part 7: Memory and State Management

Build an agent that maintains conversation history and context.

In [ ]:
# TODO: Create a conversational agent with memory

class ConversationState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    user_info: dict  # Store facts about the user
    conversation_count: int

def process_message_node(state: ConversationState) -> ConversationState:
    """Process user message and extract information."""
    messages = state["messages"]
    last_message = messages[-1].content
    user_info = state.get("user_info", {})
    
    # Extract user information
    if "my name is" in last_message.lower():
        name = last_message.lower().split("my name is")[-1].strip()
        user_info["name"] = name
    
    if "i like" in last_message.lower():
        interest = last_message.lower().split("i like")[-1].strip()
        if "interests" not in user_info:
            user_info["interests"] = []
        user_info["interests"].append(interest)
    
    return {**state, "user_info": user_info}

def generate_contextual_response_node(state: ConversationState) -> ConversationState:
    """Generate response using conversation context."""
    messages = state["messages"]
    user_info = state.get("user_info", {})
    count = state.get("conversation_count", 0) + 1
    
    last_message = messages[-1].content
    
    # Personalized response
    if user_info.get("name"):
        greeting = f"Hello {user_info['name']}! "
    else:
        greeting = "Hello! "
    
    if "interests" in user_info:
        response = greeting + f"I remember you like {', '.join(user_info['interests'])}. "
    else:
        response = greeting
    
    response += f"This is message #{count} in our conversation."
    
    return {
        **state,
        "messages": messages + [AIMessage(content=response)],
        "conversation_count": count
    }

# Build conversational agent
conv_workflow = StateGraph(ConversationState)
conv_workflow.add_node("process", process_message_node)
conv_workflow.add_node("respond", generate_contextual_response_node)

conv_workflow.set_entry_point("process")
conv_workflow.add_edge("process", "respond")
conv_workflow.add_edge("respond", END)

conv_agent = conv_workflow.compile()

print("✅ Conversational agent with memory built!")

In [ ]:
# Test conversational agent with memory
conversation_messages = [
    "Hi there!",
    "My name is Alice",
    "I like programming",
    "I also like machine learning",
    "What do you remember about me?"
]

# Maintain state across multiple turns
state = {
    "messages": [],
    "user_info": {},
    "conversation_count": 0
}

for msg in conversation_messages:
    print("\n" + "=" * 70)
    print(f"User: {msg}")
    
    # Add user message
    state["messages"] = list(state["messages"]) + [HumanMessage(content=msg)]
    
    # Run agent
    result = conv_agent.invoke(state)
    
    # Update state for next turn
    state = result
    
    # Print response
    print(f"Agent: {result['messages'][-1].content}")
    print(f"\nStored info: {result['user_info']}")

## Part 8: Challenge - Build Your Own Agent

Design and implement your own agentic system!

In [ ]:
# TODO: Design your own agent
# Ideas:
# 1. Research assistant that searches and summarizes
# 2. Task planner that breaks down projects
# 3. Code helper that debugs and suggests fixes
# 4. Customer service bot with escalation logic

# YOUR CODE HERE

print("Build your custom agent here!")
print("\nSuggested features:")
print("  - Multiple tools")
print("  - Conditional routing")
print("  - State management")
print("  - Error handling")
print("  - Conversation memory")

## Questions

Answer the following questions:

1. **What are the key differences between a simple LLM call and an agentic system?**
   - YOUR ANSWER HERE

2. **How does LangGraph's state management help build complex agents?**
   - YOUR ANSWER HERE

3. **What are the advantages and disadvantages of using tools in agents?**
   - YOUR ANSWER HERE

4. **How would you handle errors and failures in an agent system?**
   - YOUR ANSWER HERE

5. **What are some real-world applications where agentic systems would be valuable?**
   - YOUR ANSWER HERE

## Deliverables Checklist

- [ ] Understood basic agent architecture
- [ ] Built simple LangGraph workflows
- [ ] Implemented custom tools
- [ ] Created tool-using agent
- [ ] Built multi-step reasoning agent
- [ ] Implemented conversational memory
- [ ] (Optional) Designed custom agent
- [ ] Answered all questions